import numpy as np
import os

os.environ["KERAS_BACKEND"] = "jax"

In [1]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "tensorflow"
import keras

2024-03-10 10:43:44.686231: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [3]:
num_classes = 10
input_shape = (28, 28, 1)

In [4]:
model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 26, 26, 64)     │           640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 24, 24, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 12, 12, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 10, 10, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 260,298 (1016.79 KB)

 Trainable params: 260,298 (1016.79 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)

In [7]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 54s 133ms/step - acc: 0.5215 - loss: 1.3290 - val_acc: 0.9516 - val_loss: 0.1737
Epoch 2/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 51s 129ms/step - acc: 0.9222 - loss: 0.2583 - val_acc: 0.9738 - val_loss: 0.0886
Epoch 3/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 52s 129ms/step - acc: 0.9508 - loss: 0.1678 - val_acc: 0.9794 - val_loss: 0.0652
Epoch 4/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 54s 135ms/step - acc: 0.9609 - loss: 0.1298 - val_acc: 0.9829 - val_loss: 0.0544
Epoch 5/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 54s 135ms/step - acc: 0.9662 - loss: 0.1124 - val_acc: 0.9817 - val_loss: 0.0596
Epoch 6/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 53s 134ms/step - acc: 0.9714 - loss: 0.0929 - val_acc: 0.9870 - val_loss: 0.0454
Epoch 7/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 51s 127ms/step - acc: 0.9733 - loss: 0.0917 - val_acc: 0.9889 - val_loss: 0.0391
Epoch 8/20
399/399 ━━━━━━━━━━━━━━━━━━━━ 61s 154ms/step - acc: 0.9772 - loss: 0.0745 - val_acc: 0.9883 - val_loss: 0.0448
Epoch 9/20
399/399 ━━━━━━━━━━━━━

In [11]:
model.save("final_model.keras")

In [12]:
predictions = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step


In [13]:
predictions

array([[1.0575028e-09, 2.2488794e-10, 3.7471895e-05, ..., 9.9995512e-01,
        3.6257555e-10, 3.4219222e-06],
       [3.3430609e-07, 1.6842925e-09, 9.9999934e-01, ..., 1.0318359e-10,
        5.9002342e-10, 4.8740083e-12],
       [1.6343835e-06, 9.9872530e-01, 3.9374613e-06, ..., 6.4791960e-04,
        2.5679074e-05, 3.5221452e-05],
       ...,
       [4.7264957e-12, 7.5896632e-14, 7.0567376e-12, ..., 2.3437510e-16,
        7.7822609e-12, 6.4941883e-13],
       [4.6866773e-09, 4.6798335e-17, 2.6594488e-10, ..., 1.5757543e-11,
        5.4426409e-06, 7.0006462e-07],
       [3.2001335e-08, 3.8636521e-17, 5.8987895e-05, ..., 5.5988601e-22,
        1.5291258e-07, 6.5669255e-11]], dtype=float32)